In [28]:
!nvidia-smi

Tue Jan 10 05:37:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.203.03   Driver Version: 450.203.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |   1284MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 라이브러리 및 파일 불러오기

In [29]:
# load libraries

import os
import numpy as np
import pandas as pd
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/project/AIFFELTHON'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights/bc_weights')
DATA_PATH = os.path.join('data/bc')
TRAIN_PATH = os.path.join(DATA_PATH, 'front_seat_train')
TEST_PATH = os.path.join(DATA_PATH, 'front_seat_test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # connected to GPU if 'cuda' is printed here

cuda


In [ ]:
# checking imgs in a folder

for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # prints file names
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # prints img info
        plt.imshow(image)
        plt.show()
        if i==4:
            break # print 4 per folder

# Create Functions

In [32]:
# Normalize imgs, resize to 224x224
# Create pipeline
# PyTorch offers various augmentation techniques in torchvision.transforms.Compose

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

In [33]:
# creating train dataset

BATCH_SIZE = 64 # changed from 64 to 1

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  2
train:  {'11_inner_front_seat_train_2000': 0, '14_inner_sheet_dirt_train_2000': 1}


In [34]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TRAIN_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/bc/front_seat_train : 0
data/bc/front_seat_train/14_inner_sheet_dirt_train_2000 : 2000
data/bc/front_seat_train/11_inner_front_seat_train_2000 : 2000


In [35]:
# creating test dataset

BATCH_SIZE = 64 # changed from 64 to 1

test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
target_class_num = len(os.listdir(os.path.join(TEST_PATH)))

print('target_class_num: ', target_class_num)
print('test: ', _test_data.class_to_idx)

target_class_num:  2
test:  {'11_inner_front_seat_test_500': 0, '14_inner_sheet_dirt_test_500': 1}


In [36]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TEST_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/bc/front_seat_test : 0
data/bc/front_seat_test/14_inner_sheet_dirt_test_500 : 500
data/bc/front_seat_test/11_inner_front_seat_test_500 : 500


In [37]:
# metrics from sklearn.metrics

def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [38]:
# train function

def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [39]:
# test function

def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [40]:
# code to save best params based on acc

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = 'weights/bc_weights'
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'bc_classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [41]:
target_class_num

2

In [42]:
# load pre-trained resnet50

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [43]:
# result

EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.3194926611133038
Acc:  0.89575
F1:  0.8957390460835292
Precision:  0.8959163838603172
Recall:  0.89575
[[1812  188]
 [ 229 1771]]
====== Test Metrics ======
Test Loss:  0.3222171898609966
Test Acc:  0.862
Test F1:  0.8595528488381561
Test Precision:  0.8891201155751238
Test Recall:  0.862
[[365 135]
 [  3 497]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.28736963655267445
Acc:  0.9065
F1:  0.9064960494580896
Precision:  0.906568710112009
Recall:  0.9065000000000001
[[1826  174]
 [ 200 1800]]
====== Test Metrics ======
Test Loss:  0.21441101160051768
Test Acc:  0.963
Test F1:  0.9629866381763816
Test Precision:  0.9636695388140475
Test Recall:  0.963
[[491   9]
 [ 28 472]]
[Notification] Best Model Updated!
> epoch:  2

====== Training Metrics ======
Loss:  0.1796588839164802
Acc:  0.938
F1:  0.9379944039949606
Precision:  0.9381581751012116
Recall:  0.938
[[1895  10